In [1]:
import os
import torch
from tqdm import tqdm
import torchvision.transforms as transforms
import LookGenerator.datasets.transforms as custom_transforms
import numpy as np
import PIL.Image as Image
import cv2
import matplotlib.pyplot as plt
from LookGenerator.networks.segmentation import UNet
from LookGenerator.networks.utils import load_model
import albumentations
from LookGenerator.datasets.utils import load_image, prepare_image_for_segmentation

# Подготовка датасета

In [2]:
model = UNet(in_channels=3, out_channels=1)

transform_input = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.25, 0.25, 0.25]
    )
])

transform_output = transforms.Compose([
    transforms.Resize((256, 192)),
    custom_transforms.MinMaxScale(),
    custom_transforms.ThresholdTransform(threshold=0.5)
])

In [3]:
model = load_model(model, r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\segmetationBackground\weights\testResults\epoch_29.pt")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# model.eval()

cuda


In [14]:
dir_ = r"C:\Users\DenisovDmitrii\Desktop\valData"

In [15]:
def clean_image_by_indexes(array_to_clean, array):

    height, width = array.shape[0], array.shape[1]
    for i in range(height):
        for j in range(width):
            if array[i,j]==1:
                array_to_clean[i,j] = [255,255, 255]

In [16]:
list_dir = os.listdir(os.path.join(dir_, r"image"))

images = [image.split('.')[0] for image in list_dir]

In [17]:
ctr = 0
model.to(device)
for image in tqdm(images):

    real_img = load_image(dir_, "image", image, '.jpg').convert('RGB')

    real_img_as_np = np.array(real_img)
    real_img_as_np = cv2.resize(real_img_as_np, (192, 256))
    real_img_to_model = prepare_image_for_segmentation(real_img, transform = transform_input)

    modelled = model(real_img_to_model.to(device))
    modelled = transform_output(modelled.to('cpu').detach())

    torso_img = modelled.detach().numpy()[0, 0, :, :]

    clean_image_by_indexes(real_img_as_np, torso_img)
    real_img_as_np = np.array(real_img_as_np, dtype = np.uint8)

    Image.fromarray(real_img_as_np, 'RGB').save(dir_ + '\\image3\\' + image + '.png')



100%|██████████| 2032/2032 [03:30<00:00,  9.67it/s]
